In [ ]:
#!/usr/bin/env python

# Laboratorio 5: Threat Hunting with Data Science


## Parte 1 – Filtrado y preprocesamiento


### Carga y conteo de registro


In [7]:
import json

# 📌 Ruta del archivo (ajústala si es necesario)
file_path = "large_eve.json"

# 📌 Contador de registros
record_count = 0

# 📌 Cargar los datos línea por línea
with open(file_path, "r", encoding="utf-8") as f:
    for _ in f:
        record_count += 1

# 📌 Mostrar el total de registros
print(f"✅ Tráfico inicial cargado: {record_count} registros.")

✅ Tráfico inicial cargado: 746909 registros.


### Filtrar Registros DNS


In [8]:
dns_records = []

# 📌 Cargar y filtrar los datos
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            record = json.loads(line)  # Convertir línea JSON en diccionario
            if "dns" in record:  # Filtrar solo eventos DNS
                dns_records.append(record)
        except json.JSONDecodeError:
            continue  # Ignorar líneas mal formateadas

# 📌 Mostrar el total de registros DNS filtrados
print(f"✅ Se encontraron {len(dns_records)} registros DNS.")

✅ Se encontraron 15749 registros DNS.


### 2 registros aleatorios


In [9]:
import random

# 📌 Mostrar 2 registros aleatorios
if len(dns_records) >= 2:
    sample_records = random.sample(dns_records, 2)
    print("\n🔍 Ejemplo de 2 registros DNS:")
    for idx, record in enumerate(sample_records, 1):
        print(f"\n📌 Registro {idx}:")
        print(json.dumps(record, indent=4))  # Formatear JSON para mejor lectura
else:
    print("⚠️ No hay suficientes registros DNS para mostrar ejemplos.")


🔍 Ejemplo de 2 registros DNS:

📌 Registro 1:
{
    "timestamp": "2017-07-22T19:29:18.190008-0500",
    "flow_id": 1318106402055486,
    "pcap_cnt": 3352728,
    "event_type": "dns",
    "vlan": 150,
    "src_ip": "192.168.207.4",
    "src_port": 53,
    "dest_ip": "192.168.205.188",
    "dest_port": 37343,
    "proto": "UDP",
    "dns": {
        "type": "answer",
        "id": 51542,
        "rcode": "NXDOMAIN",
        "rrname": "mirror.san.fastserv.com"
    }
}

📌 Registro 2:
{
    "timestamp": "2017-07-22T19:10:22.293994-0500",
    "flow_id": 1427387475459178,
    "pcap_cnt": 2638409,
    "event_type": "dns",
    "vlan": 140,
    "src_ip": "2001:0dbb:0c18:0014:34fa:4d67:66fe:82e9",
    "src_port": 52273,
    "dest_ip": "0000:0000:0000:0000:0000:ffff:c0a8:19ee",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 22770,
        "rrname": "VERSION.BIND",
        "rrtype": "TXT",
        "tx_id": 0
    }
}


### Normalización de datos con `json_normalize`


In [10]:
import pandas as pd

# 📌 Normalizar la data JSON y convertirla en un DataFrame
df_dns = pd.json_normalize(dns_records)

# 📌 Mostrar el shape del DataFrame
print(f"✅ DataFrame normalizado con shape: {df_dns.shape}")

# 📌 Mostrar las primeras filas para inspeccionar los datos
display(df_dns.head())

✅ DataFrame normalizado con shape: (15749, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


### Filtrar registros DNS tipo A


In [14]:
# 📌 Filtrar solo registros DNS tipo A (asociados a direcciones IP)
df_dns_a = df_dns[df_dns["dns.rrtype"] == "A"]

# 📌 Mostrar el shape del DataFrame filtrado
print(f"✅ DataFrame filtrado con registros tipo A: {df_dns_a.shape}")

# 📌 Mostrar las primeras filas para verificar los datos
display(df_dns_a.head())

✅ DataFrame filtrado con registros tipo A: (2849, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN


### Filtrar dominios únicos


In [15]:
# 📌 Extraer dominios únicos de los registros DNS tipo A
unique_domains = df_dns_a["dns.rrname"].unique()

# 📌 Convertirlos en un DataFrame para visualización y análisis
df_unique_domains = pd.DataFrame(unique_domains, columns=["Unique Domains"])

# 📌 Mostrar el total de dominios únicos
print(f"✅ Se encontraron {len(df_unique_domains)} dominios únicos.")

# 📌 Mostrar las primeras filas para inspeccionar los datos
display(df_unique_domains.head())

✅ Se encontraron 177 dominios únicos.


,Unique Domains
0,api.wunderground.com
1,stork79.dropbox.com
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net
3,safebrowsing.clients.google.com.home
4,fxfeeds.mozilla.com


### TLD para los dominios


In [16]:
import tldextract


# 📌 Función para extraer el TLD de un dominio
def extract_tld(domain):
    """
    Extrae el TLD de un dominio a partir de un dominio completo.

    Parámetros:
    - domain (str): El dominio completo del cual se extraerá el TLD.

    Retorna:
    - str: El TLD extraído.
    """
    extracted = tldextract.extract(domain)

    # Si hay un sufijo, devuelve dominio + sufijo (ejemplo: wunderground.com)
    if extracted.suffix:
        return f"{extracted.domain}.{extracted.suffix}"

    # Si no hay sufijo, devuelve solo el dominio base
    return extracted.domain


# 📌 Aplicar la función a la columna 'Unique Domains'
df_unique_domains["TLD"] = df_unique_domains["Unique Domains"].apply(extract_tld)
df_unique_domains.drop("Unique Domains", axis=1, inplace=True)

# 📌 Mostrar las primeras filas con los TLD extraídos
print("✅ TLDs extraídos correctamente.")
display(df_unique_domains.head())

✅ TLDs extraídos correctamente.


,TLD
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,home
4,mozilla.com


## Parte 2 - Data Science


### Gemini


#### Configuración de Gemini y carga de la API Key


In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

# 📌 Obtener la ruta actual del directorio
dotenv_path = os.path.join(os.getcwd(), ".env.local")

print(f"🔍 Loading dotenv from: {dotenv_path}")

# 📌 Cargar las variables del archivo .env.local
load_dotenv(dotenv_path=dotenv_path, override=True)

# 📌 Verificar si la API Key se cargó correctamente
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("❌ ERROR: No se encontró una API Key válida en .env.local")
    exit()

print(f"✅ API Key cargada correctamente: {api_key[:5]}")

🔍 Loading dotenv from: /Users/mvrcentes/Library/CloudStorage/OneDrive-UVG/Documentos/Semestre_9/Security_Data_Science/Security_Data_Science/.env.local
✅ API Key cargada correctamente: AIzaS


/Users/mvrcentes/Library/CloudStorage/OneDrive-UVG/Documentos/Semestre_9/Security_Data_Science/Security_Data_Science/myvenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Función para clasificar dominios con Gemini


In [2]:
import google.generativeai as genai
import os
import time


def classify_domain_with_gemini(domain, max_retries=3, wait_time=2):
    """
    Clasifica un dominio como DGA (1) o legítimo (0) usando Gemini AI.

    Parámetros:
    - domain (str): El dominio a evaluar.
    - max_retries (int): Número máximo de intentos en caso de error.
    - wait_time (int): Tiempo en segundos entre intentos.

    Retorna:
    - int: 1 si el dominio es DGA, 0 si es legítimo.
    """
    for attempt in range(1, max_retries + 1):
        try:
            # 📌 Configurar la API Key
            genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

            # 📌 Usar Gemini Pro para mayor precisión
            model = genai.GenerativeModel("gemini-1.5-flash-latest")

            # 📌 Nuevo prompt mejorado
            prompt = f"""
            Eres un experto en ciberseguridad y análisis de dominios.

            Tu tarea es clasificar el siguiente dominio en una de estas dos categorías:
            1️⃣ **DGA (1)**: Un dominio generado por un algoritmo malicioso (Domain Generation Algorithm). 
               - Suelen ser nombres aleatorios sin sentido (ej: xjklzqw.net, a8f4h3d2.com).
               - Se usan en malware para comunicación con servidores de comando y control.

            0️⃣ **Legítimo (0)**: Un dominio de una empresa, sitio web o servicio popular.
               - Ejemplos: google.com, dropbox.com, amazon.com, microsoft.com.
               - Si el dominio pertenece a una empresa reconocida o servicio conocido, clasifícalo como 0.

            Dominio a analizar: {domain}

            📌 **Importante:**
            - **Solo responde con el número 1 o 0.**
            - No agregues explicaciones adicionales.
            - Si el dominio parece legítimo, responde "0".
            """

            # 📌 Consultar Gemini
            response = model.generate_content(prompt)

            # 📌 Extraer y validar la respuesta
            label = response.text.strip()
            if label in ["0", "1"]:
                return int(label)

            print(f"⚠️ Respuesta inesperada de Gemini en intento {attempt}: {label}")

        except Exception as e:
            error_message = str(e)
            print(
                f"⚠️ Intento {attempt}/{max_retries} - Error clasificando {domain}: {error_message}"
            )

            # 📌 Si se agota la cuota, detener clasificación
            if "429" in error_message or "Resource has been exhausted" in error_message:
                print(
                    "⏳ Intentar más tarde, cuota de Gemini agotada. Clasificación detenida."
                )
                return None

        # 📌 Esperar antes de reintentar
        time.sleep(wait_time)

    return None  # Si todos los intentos fallan

#### Cargar los datos a un CSV


In [3]:
import pandas as pd
import os

# 📌 Nombre del archivo CSV donde se almacenarán las clasificaciones
csv_filename = "classified_domains.csv"

# 📌 Verificar si existe el archvio o empezar desde 0
if os.path.exists(csv_filename):
    # 📌 Cargar el archivo CSV existente
    df_existing = pd.read_csv(csv_filename)

    # 📌 Mostrar las primeras filas del archivo existente
    print(f"✅ Archivo CSV existente cargado con {df_existing.shape[0]} registros.")
    display(df_existing.head())
else:
    # 📌 Crear un DataFrame vacío con las columnas necesarias
    df_existing = pd.DataFrame(columns=["domain_tld", "dga_label"])
    print("🆕 Creando el archivo CSV desde cero.")

    # 📌 Guardar el archivo vacío para inicializarlo
    df_existing.to_csv(csv_filename, index=False)

✅ Archivo CSV existente cargado con 177 registros.


,domain_tld,dga_label
0,wunderground.com,0
1,dropbox.com,0
2,aoltw.net,1
3,home,[missing]
4,mozilla.com,[missing]


In [ ]:
# 📌 Extraer los dominios únicos desde df_unique_domains
df_new_domains = df_unique_domains[["TLD"]].rename(columns={"TLD": "domain_tld"})

# 📌 Agregar la columna 'dga_label' con valores `[missing]`
df_new_domains["dga_label"] = "[missing]"

# 📌 Guardar los dominios en el CSV
df_new_domains.to_csv(csv_filename, index=False)

print(
    f"✅ Archivo {csv_filename} guardado correctamente con {len(df_new_domains)} dominios."
)

✅ Archivo classified_domains.csv guardado correctamente con 177 dominios.


### Cargar el CSV y verificar los dominios sin clasificar


In [3]:
import pandas as pd

# 📌 Nombre del archivo CSV con los dominios
csv_filename = "classified_domains.csv"

# 📌 Cargar el CSV
df_existing = pd.read_csv(csv_filename)

# 📌 Verificar cuántos dominios necesitan ser clasificados
to_classify = df_existing[df_existing["dga_label"] == "[missing]"]["domain_tld"]

print(f"🔍 Se encontraron {len(to_classify)} dominios sin clasificar.")
display(df_existing.head())  # Mostrar los primeros registros

🔍 Se encontraron 89 dominios sin clasificar.


,domain_tld,dga_label
0,wunderground.com,0
1,dropbox.com,0
2,aoltw.net,1
3,home,0
4,mozilla.com,0


### Clasificar los dominios


In [4]:
from tqdm import tqdm

# 📌 Bandera para detectar si se agotó la cuota
quota_exceeded = False

# 📌 Clasificar solo los dominios sin etiqueta
for domain in tqdm(to_classify, desc="Clasificando dominios con Gemini"):
    if quota_exceeded:
        print("⚠️ Se ha agotado la cuota de Gemini. Deteniendo clasificación.")
        break  # Salir del bucle inmediatamente

    classification = classify_domain_with_gemini(domain)

    # 📌 Si se agotó la cuota, detener todo
    if classification is None:
        quota_exceeded = True
        print("⏳ Intentar más tarde, cuota de Gemini agotada. Clasificación detenida.")
        break

    # 📌 Actualizar el DataFrame con la nueva clasificación
    df_existing.loc[df_existing["domain_tld"] == domain, "dga_label"] = classification

    # 📌 Guardar el CSV después de cada clasificación para no perder progreso
    df_existing.to_csv(csv_filename, index=False)

print(f"✅ Clasificación finalizada. Verifica el archivo {csv_filename}.")

Clasificando dominios con Gemini:   0%|          | 0/89 [00:00<?, ?it/s]

Clasificando dominios con Gemini:  18%|█▊        | 16/89 [00:15<01:12,  1.01it/s]

⚠️ Intento 1/3 - Error clasificando securityfocus.com: 429 Resource has been exhausted (e.g. check quota).
⏳ Intentar más tarde, cuota de Gemini agotada. Clasificación detenida.
⏳ Intentar más tarde, cuota de Gemini agotada. Clasificación detenida.
✅ Clasificación finalizada. Verifica el archivo classified_domains.csv.


In [1]:
import os
import sys
from datetime import datetime

import numpy as np
import requests


def ola(a, b):
    print("a + b")
    return a + b


# sys.path.append(
#     os.path.abspath(os.path.join(os.path.dirname(__file__), os.path.pardir))
# )
# datetime.now()
# np.random.rand(5)
# requests.get("https://www.google.com")


ModuleNotFoundError: No module named 'numpy'